In [2]:
import pandas as pd
import polars as pl

In [13]:
pd.read_csv("/home/zorinep/big_data_magistracy/technology/arXiv_scientific dataset.csv").columns

Index(['id', 'title', 'category', 'category_code', 'published_date',
       'updated_date', 'authors', 'first_author', 'summary',
       'summary_word_count'],
      dtype='object')

# Скорость чтения

In [15]:
%%time
df_pandas = pd.read_csv("/home/zorinep/big_data_magistracy/technology/arXiv_scientific dataset.csv")
df_pandas.head()

CPU times: user 2.98 s, sys: 282 ms, total: 3.26 s
Wall time: 3.2 s


,id,title,category,category_code,published_date,updated_date,authors,first_author,summary,summary_word_count
0,cs-9308101v1,Dynamic Backtracking,Artificial Intelligence,cs.AI,8/1/93,8/1/93,['M. L. Ginsberg'],'M. L. Ginsberg',Because of their occasional need to return to ...,79
1,cs-9308102v1,A Market-Oriented Programming Environment and ...,Artificial Intelligence,cs.AI,8/1/93,8/1/93,['M. P. Wellman'],'M. P. Wellman',Market price systems constitute a well-underst...,119
2,cs-9309101v1,An Empirical Analysis of Search in GSAT,Artificial Intelligence,cs.AI,9/1/93,9/1/93,"['I. P. Gent', 'T. Walsh']",'I. P. Gent',We describe an extensive study of search in GS...,167
3,cs-9311101v1,The Difficulties of Learning Logic Programs wi...,Artificial Intelligence,cs.AI,11/1/93,11/1/93,"['F. Bergadano', 'D. Gunetti', 'U. Trinchero']",'F. Bergadano',As real logic programmers normally use cut (!)...,174
4,cs-9311102v1,Software Agents: Completing Patterns and Const...,Artificial Intelligence,cs.AI,11/1/93,11/1/93,"['J. C. Schlimmer', 'L. A. Hermens']",'J. C. Schlimmer',To support the goal of allowing users to recor...,187


In [21]:
%%time
df_polars = pl.read_csv("/home/zorinep/big_data_magistracy/technology/arXiv_scientific dataset.csv")
df_pandas.head()

CPU times: user 662 ms, sys: 52.8 ms, total: 714 ms
Wall time: 107 ms


,id,title,category,category_code,published_date,updated_date,authors,first_author,summary,summary_word_count
0,cs-9308101v1,Dynamic Backtracking,Artificial Intelligence,cs.AI,8/1/93,8/1/93,['M. L. Ginsberg'],'M. L. Ginsberg',Because of their occasional need to return to ...,79
1,cs-9308102v1,A Market-Oriented Programming Environment and ...,Artificial Intelligence,cs.AI,8/1/93,8/1/93,['M. P. Wellman'],'M. P. Wellman',Market price systems constitute a well-underst...,119
2,cs-9309101v1,An Empirical Analysis of Search in GSAT,Artificial Intelligence,cs.AI,9/1/93,9/1/93,"['I. P. Gent', 'T. Walsh']",'I. P. Gent',We describe an extensive study of search in GS...,167
3,cs-9311101v1,The Difficulties of Learning Logic Programs wi...,Artificial Intelligence,cs.AI,11/1/93,11/1/93,"['F. Bergadano', 'D. Gunetti', 'U. Trinchero']",'F. Bergadano',As real logic programmers normally use cut (!)...,174
4,cs-9311102v1,Software Agents: Completing Patterns and Const...,Artificial Intelligence,cs.AI,11/1/93,11/1/93,"['J. C. Schlimmer', 'L. A. Hermens']",'J. C. Schlimmer',To support the goal of allowing users to recor...,187


In [24]:
%%time
df_lazy = pl.scan_csv("/home/zorinep/big_data_magistracy/technology/arXiv_scientific dataset.csv")
df_lazy.head()

CPU times: user 573 μs, sys: 0 ns, total: 573 μs
Wall time: 584 μs


# Время запроса

In [20]:
%%time
category_percent_pandas = df_pandas['category'].value_counts(normalize=True) * 100
category_percent_pandas.head(20)

CPU times: user 10.4 ms, sys: 36 μs, total: 10.4 ms
Wall time: 9.29 ms


category
Machine Learning                                          29.350108
Computer Vision and Pattern Recognition                   21.328117
Computation and Language (Natural Language Processing)    18.498510
Artificial Intelligence                                    9.519371
Machine Learning (Statistics)                              7.668198
Neural and Evolutionary Computing                          4.043659
Robotics                                                   0.662811
Information Retrieval                                      0.661343
Methodology (Statistics)                                   0.647396
Computation and Language (Legacy category)                 0.550507
Optimization and Control                                   0.549773
Cryptography and Security                                  0.488116
Computers and Society                                      0.378015
Social and Information Networks                            0.306082
Human-Computer Interaction             

In [23]:
%%time
category_percent_polars = (
    df_polars
    .group_by('category')
    .agg(pl.count().alias('count'))
    .with_columns(
        (pl.col('count') / pl.col('count').sum() * 100).alias('percentage')
    )
    .sort('percentage', descending=True)
)
category_percent_polars.head(20)

CPU times: user 36.6 ms, sys: 10.2 ms, total: 46.8 ms
Wall time: 12.2 ms


<timed exec>:4: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)


category,count,percentage
str,u32,f64
"""Machine Learning""",39986,29.350108
"""Computer Vision and Pattern Re…",29057,21.328117
"""Computation and Language (Natu…",25202,18.49851
"""Artificial Intelligence""",12969,9.519371
"""Machine Learning (Statistics)""",10447,7.668198
…,…,…
"""Signal Processing""",405,0.297274
"""Software Engineering""",382,0.280392
"""Neurons and Cognition""",362,0.265711


In [25]:
%%time
category_percent_lazy = (
    df_lazy
    .group_by('category')
    .agg(pl.count().alias('count'))
    .with_columns(
        (pl.col('count') / pl.col('count').sum() * 100).alias('percentage')
    )
    .sort('percentage', descending=True)
)
result_percent = category_percent_lazy.collect()
result_percent.head(20)

CPU times: user 547 ms, sys: 70.8 ms, total: 618 ms
Wall time: 169 ms


<timed exec>:4: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)


category,count,percentage
str,u32,f64
"""Machine Learning""",39986,29.350108
"""Computer Vision and Pattern Re…",29057,21.328117
"""Computation and Language (Natu…",25202,18.49851
"""Artificial Intelligence""",12969,9.519371
"""Machine Learning (Statistics)""",10447,7.668198
…,…,…
"""Signal Processing""",405,0.297274
"""Software Engineering""",382,0.280392
"""Neurons and Cognition""",362,0.265711


# Подсчет общего количества слов по категориям

In [27]:
%%time
words_by_category_pandas = df_pandas.groupby('category')['summary_word_count'].sum().reset_index()
words_by_category_pandas = words_by_category_pandas.sort_values('summary_word_count', ascending=False)
words_by_category_pandas.head(20)

CPU times: user 23 ms, sys: 147 μs, total: 23.1 ms
Wall time: 18.9 ms


,category,summary_word_count
79,Machine Learning,6663792
31,Computer Vision and Pattern Recognition,4970330
24,Computation and Language (Natural Language Pro...,3809155
7,Artificial Intelligence,2012783
80,Machine Learning (Statistics),1621620
93,Neural and Evolutionary Computing,905486
73,Information Retrieval,155726
119,Robotics,151603
87,Methodology (Statistics),141734
34,Cryptography and Security,123272


In [29]:
%%time
words_by_category_polars = (
    df_polars
    .group_by('category')
    .agg([
        pl.sum('summary_word_count').alias('total_words'),
        pl.count().alias('article_count')
    ])
    .sort('total_words', descending=True)
)
words_by_category_polars.head(20)

CPU times: user 33.7 ms, sys: 12.4 ms, total: 46.1 ms
Wall time: 11.7 ms


<timed exec>:6: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)


category,total_words,article_count
str,i64,u32
"""Machine Learning""",6663792,39986
"""Computer Vision and Pattern Re…",4970330,29057
"""Computation and Language (Natu…",3809155,25202
"""Artificial Intelligence""",2012783,12969
"""Machine Learning (Statistics)""",1621620,10447
…,…,…
"""Software Engineering""",70830,382
"""Human-Computer Interaction""",66254,411
"""Neurons and Cognition""",64255,362


In [30]:
%%time
words_by_category_lazy = (
    df_lazy
    .group_by('category')
    .agg([
        pl.sum('summary_word_count').alias('total_words'),
        pl.count().alias('article_count')
    ])
    .sort('total_words', descending=True)
)
result_words = words_by_category_lazy.collect()
result_words.head(20)

CPU times: user 167 ms, sys: 59.2 ms, total: 226 ms
Wall time: 82.7 ms


<timed exec>:6: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)


category,total_words,article_count
str,i64,u32
"""Machine Learning""",6663792,39986
"""Computer Vision and Pattern Re…",4970330,29057
"""Computation and Language (Natu…",3809155,25202
"""Artificial Intelligence""",2012783,12969
"""Machine Learning (Statistics)""",1621620,10447
…,…,…
"""Software Engineering""",70830,382
"""Human-Computer Interaction""",66254,411
"""Neurons and Cognition""",64255,362


# Извлечение года из даты и группировка

In [ ]:
%%time
df_pandas['published_date'] = pd.to_datetime(df_pandas['published_date'])
df_pandas['year'] = df_pandas['published_date'].dt.year

words_by_year_pandas = (
    df_pandas
    .groupby('year')
    .agg({
        'summary_word_count': 'sum',
        'id': 'count'
    })
    .rename(columns={'summary_word_count': 'total_words', 'id': 'article_count'})
    .sort_values('year')
    .reset_index()
)
words_by_year_pandas.head(20)

<timed exec>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.


CPU times: user 601 ms, sys: 4.57 ms, total: 606 ms
Wall time: 556 ms


,year,total_words,article_count
0,1993,887,6
1,1994,18179,175
2,1995,27736,251
3,1996,26405,229
4,1997,22010,182
5,1998,8150,66
6,1999,9048,74
7,2000,24026,210
8,2001,12023,106
9,2002,18914,151


In [39]:
%%time
words_by_year_polars = (
    df_polars
    .with_columns(
        pl.col('published_date')
        .str.strptime(pl.Date, format="%m/%d/%y")  
        .dt.year()
        .alias('year')
    )
    .group_by('year')
    .agg([
        pl.sum('summary_word_count').alias('total_words'),
        pl.count().alias('article_count')
    ])
    .sort('year')
)
words_by_year_polars.head(20)

CPU times: user 49.7 ms, sys: 13.3 ms, total: 63 ms
Wall time: 15.9 ms


<timed exec>:12: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)


year,total_words,article_count
i32,i64,u32
1993,887,6
1994,18179,175
1995,27736,251
1996,26405,229
1997,22010,182
…,…,…
2008,56548,447
2009,79381,616
2010,105279,749


In [ ]:
%%time
words_by_year_lazy = (
    df_lazy
    .with_columns(
        pl.col('published_date')
        .str.strptime(pl.Date, format="%m/%d/%y") 
        .dt.year()
        .alias('year')
    )
    .group_by('year')
    .agg([
        pl.sum('summary_word_count').alias('total_words'),
        pl.count().alias('article_count')
    ])
    .sort('year')
)
words_by_year_lazy.head(20)

CPU times: user 567 μs, sys: 66 μs, total: 633 μs
Wall time: 558 μs


<timed exec>:12: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
